# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [45]:
from mxnet import ndarray as nd
from mxnet import context
import time
import numpy as np
def try_gpu(i):
    if context.num_gpus() < i+1:
        print('-'*20 + 'sorry, you can not use gpu' + '-'*20)
        return context.cpu()
    return context.gpu(i)

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [32]:
def matmul(A, B, mul_type='matrix-matrix', context_type=context.cpu()):
    n1, n2, n3 = A.shape[0], A.shape[1], B.shape[1]
    C = nd.zeros((n1, n3), ctx=context_type)
    tic = time.time()
    
    if mul_type == 'matrix-matrix':
        C = nd.dot(A, B)
    elif mul_type == 'matrix-vector':
        for i in range(n3):
            C[: , i] = nd.dot(A, B[: , i])
    elif mul_type == 'vector-vector':
        for i in range(n1):
            for j in range(n3):
                C[i][j] = nd.dot(A[i, : ], B[: , j])
                
    C.wait_to_read()
    used_time = time.time() - tic
    
    return C, used_time

In [33]:
size = 100
A_orignial = nd.random.randn(size, size)
B_orignial = nd.random.randn(size, size)
mul_types = ['matrix-matrix', 'matrix-vector', 'vector-vector']

for use_gpu in [False, True]:
    if use_gpu:
        context_type = try_gpu(0)
        A, B = A_orignial.copyto(context_type), B_orignial.copyto(context_type)
    else:
        context_type = context.cpu()
        A, B = A_orignial, B_orignial
        
    for mul_type in mul_types:
        C, used_time = matmul(A, B, mul_type=mul_type, context_type=context_type)
        print('compute A*B with {0}: {1}'.format(mul_type, round(used_time, 6)))

compute A*B with matrix-matrix: 0.000959
compute A*B with matrix-vector: 0.031052
compute A*B with vector-vector: 2.98823
--------------------sorry, you can not use gpu--------------------
compute A*B with matrix-matrix: 0.000105
compute A*B with matrix-vector: 0.020026
compute A*B with vector-vector: 2.92375


## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

$$
\begin{align}
1.证明：
&\because B^T = AD^TA^T = ADA^T = B\\
&\therefore B是对称矩阵\\
&设x为任意非0向量，显然\hat x=xA\neq0\\
&\because D为非负对角矩阵\\
&\therefore \hat xD\hat x^T\geq0,即xBx^T\geq0\\
&综上，B为半正定矩阵
\end{align}	
$$				


第2问参考答案：

It would be more useful to work with B if m << n as matrix multiplication for instance for two arbitrary matrices X and Y with dimesnions a by b and b by c is a runtime of O(abc). If we multiply B by a matrix C that is m by k, then BC is computed in O(m2k). This matrix multiplication with ADATC is O(2mnk + n2k). Thus if m << n, then it would be more efficient to use B and if n >> m, then it would be better to use A and D.

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [44]:
!nvidia-smi
x = nd.ones((2, 2), ctx=try_gpu(0))
x

/bin/sh: nvidia-smi: command not found
--------------------sorry, you can not use gpu--------------------



[[1. 1.]
 [1. 1.]]
<NDArray 2x2 @cpu(0)>

## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [62]:
def exp_1(A, B):
    n = A.shape[0]
    c = np.zeros(n)
    tic = time.time()
    for i in range(n):
        c[i] = np.linalg.norm(np.dot(A.asnumpy(), B[: , i].asnumpy()))
        #c[i] = (nd.norm(nd.dot(A, B[: , i])).asscalar())**2
    return c, time.time()-tic

def exp_2(A, B):
    n = A.shape[0]
    c = nd.zeros(n)
    tic = time.time()
    for i in range(n):
        c[i] = (nd.norm(nd.dot(A, B[: , i])))**2
    c = c.asnumpy()
    return c, time.time()-tic
size = 1000
A, B = nd.random.randn(size, size), nd.random.randn(size, size)
_, used_time1 = exp_1(A, B)
_, used_time2 = exp_2(A, B)
print('experiment 1 used time: {0}\nexperiment 2 used time; {1}'.format(used_time1, used_time2))

experiment 1 used time: 0.4060671329498291
experiment 2 used time; 0.2716867923736572


不是很理解上面的问题，如果按照自己的做法，可以看出从NDarray转化到np.array是会花费不少代价的

看完答案后没有解释这题的目的，不过答案用的是上面注释掉的代码，另外在网上看到一个[链接](https://mxnet.incubator.apache.org/versions/master/tutorials/python/profiler.html)，关于NDarray转化到np.array，以及刻画模型运行指标的

## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [59]:
A = nd.ones((10, 10))
B = nd.ones((10, 10))
C = nd.ones((10, 10))
C += nd.dot(A, B)

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [57]:
x = nd.ones((10, 1))
y = nd.random.randn(1, 10)
(y**x)


[[ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50812554 -1.2068808   1.2174315   0.13453278 -0.7020151  -2.2163332
  -0.48276684  0.5110155  -1.6071362  -2.9807727 ]
 [ 0.50